In [1]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp agents.dqn.targets

In [4]:
# export
# Python native modules
import os
from collections import deque
from typing import *
# Third party libs
import torch
from torch.nn import *
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.data.block import *
from fastrl.data.gym import *
from fastrl.agent import *
from fastrl.core import *
from fastrl.agents.dqn.core import *
from fastrl.memory.experience_replay import *

# DQN Targets + N-Step
> A Bare-Bones DQN is usually extremely unstable. Target models can eleviate this. We also support First-Last N steps better.

In [5]:
GAMMA=0.99
def calc_target(net, local_reward, next_state,d):
    if next_state is None or d:
        return local_reward
    state_v = torch.tensor([next_state], dtype=torch.float32).to(default_device())
    next_q_v = net(state_v)
    best_q = next_q_v.max(dim=1)[0].item()
    return local_reward + GAMMA * best_q



In [208]:
# for  i in range(1000):

# learn.opt.zero_grad()

learn.state_action_values = learn.model.model(learn.xb['state']).gather(1, learn.xb['action']).squeeze(-1)
learn.next_state_values = learn.target_model(learn.xb['next_state']).max(1)[0]
learn.next_state_values[learn.xb['done'].squeeze(-1)] = 0.0

learn.expected_state_action_values = learn.next_state_values.detach() * (0.99**3) + learn.xb['reward'].squeeze(-1)
learn.loss= nn.MSELoss()(learn.state_action_values,learn.next_state_values)

# learn.loss.backward()
# learn.opt.step()

print(learn.loss)

TensorBatch(0.0069, device='cuda:0', grad_fn=<AliasBackward>)


In [209]:
learn.state_action_values

TensorBatch([0.0811, 0.1566, 0.0742, 0.1281, 0.0266, 0.1624, 0.1761, 0.0811, 0.0742,
        0.0266, 0.1566, 0.1624, 0.1796, 0.1624, 0.1515, 0.1479, 0.1796, 0.1566,
        0.0266, 0.1271, 0.1481, 0.1926, 0.1119, 0.1026, 0.0591, 0.1477, 0.0266,
        0.0591, 0.1471, 0.1481, 0.1749, 0.1941], device='cuda:0',
       grad_fn=<AliasBackward>)

In [201]:
learn.xb['reward'].squeeze(-1)

TensorBatch([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0')

In [179]:
learn.next_state_values.detach()

tensor([13.6296, 15.6673, 12.8571, 14.1879, 12.4095,  9.9496, 11.5220,  9.8726,
        10.6655,  0.0000,  5.8960, 10.1341, 11.9803, 13.1301, 12.9792, 15.0890,
        12.9226, 15.4550,  0.0000, 12.4242,  0.0000, 14.3504, 11.0739,  8.9833,
        15.3750, 13.9025, 10.4124, 12.0961,  0.0000, 11.4348, 15.0265, 13.6760],
       device='cuda:0')

In [202]:
# export
class DQNTargetTrainer(Callback):
    
    def __init__(self,n_batch=0,target_sync=300,discount=0.99,n_steps=1):
        store_attr()
        self._xb=None  
        
    def before_fit(self):
        self.learn.target_model=deepcopy(self.learn.model.model)
        self.n_batch=0
    
    def after_pred(self):
        self._xb=self.yb
        self.learn.yb=[]
        

  
        self.learn.opt.zero_grad()

        self.learn.state_action_values = self.learn.model.model(self.learn.xb['state']).gather(1, self.learn.xb['action']).squeeze(-1)
        self.learn.next_state_values = self.target_model(self.xb['next_state']).max(1)[0]
        self.learn.next_state_values[self.xb['done'].squeeze(-1)] = 0.0

        self.learn.expected_state_action_values = self.learn.next_state_values.detach() * (self.discount**self.n_steps) + self.xb['reward'].squeeze(-1)
        self.learn.loss= nn.MSELoss()(self.learn.state_action_values,self.learn.expected_state_action_values)
        
        if (self.n_batch-1)%self.target_sync==0:
            print('The loss should be practically zero: ',self.loss)
        
        raise Exception
        self.learn.loss.backward()
        self.learn.opt.step()
#         self.learn.batch_targets = torch.cat([calc_target(self.learn.model.model, r, ns.cpu().numpy(),d)
#                          for r,ns,d in zip(self.learn.xb['reward'],self.learn.xb['next_state'],self.learn.xb['done'])])
        
#         self.learn.opt.zero_grad()
#         self.learn.states_v = self.xb['state'].to(default_device()).float()
#         self.learn.net_q_v = self.learn.model.model(self.learn.states_v)
#         # print(net_q_v)
#         self.learn.target_q = self.learn.net_q_v.cpu().data.numpy().copy()
        
#         # print(batch_targets,target_q)
#         self.learn.target_q[range(self.learn.xb.bs()), self.xb['action'].cpu()] = self.learn.batch_targets.cpu()
#         self.learn.target_q_v = torch.tensor(self.learn.target_q)
#         # print(net_q_v, target_q_v)
#         loss_v = self.learn.loss_func(self.learn.net_q_v.cpu(), self.learn.target_q_v.cpu())
#         loss_v.backward()
#         self.learn.loss=loss_v.cpu()
#         # print(loss_v)
#         self.learn.opt.step()
        
#         self.learn.yb=self.xb
#         self.learn.xb=self.xb[0]
#         self._xb=({k:v.clone() for k,v in self.xb.items()},)
#         self.learn.done_mask=self.xb['done'].reshape(-1,)
#         self.learn.next_q=self.target_model(self.xb['next_state']).max(dim=1).values.reshape(-1,1)
#         self.learn.next_q[self.done_mask]=1
#         self.learn.targets=self.xb['reward']+self.learn.next_q*(self.discount**self.n_steps)
#         self.learn.pred=self.learn.model.model(self.xb['state'])

#         t_q=self.pred.clone()
#         t_q.scatter_(1,self.xb['action'],self.targets)
#         self.learn.yb=(t_q,)
        with torch.no_grad():
            self.learn.td_error=(self.learn.state_action_values.cpu()-self.learn.expected_state_action_values.cpu()).reshape(-1,1)**2
        
    def before_backward(self): self.learn.yb=self._xb
        
    def after_batch(self):
        if self.n_batch%self.target_sync==0:
            self.target_model.load_state_dict(self.learn.model.state_dict())
            # if self.n_batch>1:raise Exception
        self.n_batch+=1

In [227]:
import torch.optim as optim

In [253]:
from fastai.learner import OptimWrapper

In [254]:
OptimWrapper?

Init signature: OptimWrapper(params, opt, hp_map=None, convert_groups=True, **kwargs)
Docstring:      A wrapper class for existing PyTorch optimizers
File:           ~/fastai/fastai/optimizer.py
Type:           type
Subclasses:     


In [259]:
dqn=DQN(4,2)
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect(min_epsilon=0.02)])
source=Source(cbs=[GymLoop('CartPole-v1',agent,steps_count=1,seed=None,#mode='rgb_array',
                           steps_delta=1),FirstLast#,ResReduce(reduce_by=4)
                  ])
dls=SourceDataBlock().dataloaders([source],n=1000,bs=1,num_workers=0)

# er_tb=ExperienceReplayTensorboard(comment='_dqn_target',every_epoch=1)
# opt=optim.Adam(dqn.parameters(), lr=0.0001)
learn=Learner(dls,agent,loss_func=MSELoss(),
              opt_func=partial(OptimWrapper,dqn.parameters(),optim.Adam),
              cbs=[ExperienceReplayCallback(bs=32,max_sz=1000,warmup_sz=32,freeze_at_max=True),
                   DQNTargetTrainer(n_steps=1)
                   # ,er_tb
                  ],
              metrics=[Reward,Epsilon,NEpisodes])

Could not do one pass in your dataloader, there is something wrong in it


In [260]:
# hide
SHOW_TENSOR_BOARD=False
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard(samples_per_plugin='images=2000')

A few things, the loss should be <1. If it is not, there is something majorly wrong iwth the training.

It is alright for the actual values twe are comparing to be way more than >1 though, but it is expected that the loss should not be all that high at all.

In [262]:
slow=True
learn.fit(3 if not slow else 47,lr=0.0001,wd=None)

RuntimeError: CUDA error: unspecified launch failure

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()